# CHEM 1000 - Spring 2024
Prof. Geoffrey Hutchison, University of Pittsburgh

## 0. Introduction

**Learning Objectives**

By the end of this session, you should be able to:
- Use the `pint` module in Python to convert units
- Solve most unit conversion problems known to humans

### 0.b Unit Conversion

Unit conversion is usually a major annoyance in physical sciences. Fortunately, it's possible in Python to track and convert units using a package called `pint` (hah.)

`pint` tracks a [lot of units](https://github.com/hgrecco/pint/blob/master/pint/default_en.txt) and most standard prefixes (e.g., *pico-* to *giga-* and more).

Here's a quick tutorial - there's more [documentation online](https://pint.readthedocs.io/en/stable/index.html) including a more complete [tutorial](https://pint.readthedocs.io/en/stable/tutorial.html).

In [ ]:
# this is some code for this class to make sure we have up-to-date modules installed
import sys
!{sys.executable} -m pip install pint --upgrade --user

In [ ]:
import pint

ureg = pint.UnitRegistry()

In [ ]:
# we can convert this
car_speed = 55*ureg.miles/ureg.hour
print(car_speed.to('km/hr'))
print(car_speed.to_base_units())

# alternately
Q_ = ureg.Quantity
Q_(55, 'miles/hour').to_base_units()

These look like too many significant figures. We can use `format(number, '.6f')` or `round(number, digits)` to clean that up:

In [ ]:
meters_per_sec = Q_(55, 'miles/hour').to_base_units()
format(meters_per_sec, '.0f') # 55 has 2 significant figures, so no decimal places

I'm considering importing a car - it claims 5.0 L/100km fuel efficiency, but I know miles per gallon...

In [ ]:
ureg.define('mpg = 1 * mile / gallon')
ureg.define('_100km = 100 * kilometer')
fuel_ec_europe = 5 * ureg.L / ureg._100km

(1 / fuel_ec_europe).to(ureg.mpg)

In [ ]:
# Let's give a more reasonable accuracy
mpg = (1 / fuel_ec_europe).to(ureg.mpg)
format(mpg, '.0f') # 2 significant figures in 5.0 L / 100km

In [ ]:
light = Q_("299792458 m/s")
print(format(light.to("miles/hour"), '.0f'))

Okay, I think that breaks the speed limit around here...

In [ ]:
print(light.to("miles/second")) # too many sig figs, but we'll skip that for now

In [ ]:
# ooh, let's figure out the lightning flash thing..
# essentially, light travels so fast, that it's effectively instantaneous
time = 1.0*ureg.mile / light
print('light 1 mile in: ', time.to_base_units().to_compact())

# so let's figure out how far the sound travels in a second
# Wikipedia gives ~346 m/s on a summer day: https://en.wikipedia.org/wiki/Speed_of_sound#Tables
sound = Q_(346, 'm/s')
print(sound.to('miles/s'))

So, if you count ~5 seconds between the flash and the thunder, the lighting is 1 mile away

In [ ]:
# some math, with conversion between mixed units
floor_area = 12*ureg.feet * 10*ureg.meter
floor_area.to_base_units()

In [ ]:
# what about density?
# aka, do I float?

weight = 165*ureg.lbs
height = 6*ureg.ft + 2*ureg.inches
width = 18*ureg.inches # honestly, I have no idea
depth = 5*ureg.inches # a random guess

volume = height * width * depth # well, no, but let's assume I'm a box
density = weight / volume
print(density.to_base_units())
print(density.to('g/mL'))

In [ ]:
# temperature conversion
weather = Q_(82, "degF") # degrees Fahrenheit so we don't need to find the ° character
ureg.default_format = '.2f' # number of decimals
print(weather.to('degC')) # degrees C
print(weather.to('kelvin'))

Batteries are often listed with somewhat weird capacity units - milli-Ampere•hour (e.g., they can supply an electrical current of 1 mA for X number of hours. Here's my iPhone:

In [ ]:
capacity = Q_('3110 mA*hr')
print(capacity)

The weird thing is that electrical current is charge $\times$ time, so we should be able to convert that to Coulombs (i.e., units of charge):

In [ ]:
capacity.to('coulomb')

Let's create a new unit - so we can figure out how many electrons are in my iPhone battery...

In [ ]:
ureg.define('charge = 1.0/6.2415090744e18 coulomb')
print(capacity.to('charges'))

print('moles ', capacity.to('charges').magnitude / 6.023e23)

A little over 0.1 mole, which seems fairly reasonable. We could, if we wanted, work out charge density (volume) or charg density per gram, etc.

Here's a personal favorite. The atomic unit of electrical field corresponds to the field between a proton and an electron in a hydrogen atom.

In [ ]:
au = Q_('1.0 atomic_unit_of_electric_field')
# the "to_compact" method will add prefixes as needed
au.to_compact('V/m')

In [ ]:
au.to('V/nm')

That's obviously pretty big. What if we have one charge (e.g. a Na<sup>+</sup> ion) at one nanometer from a molecule:

In [ ]:
ion_field = Q_(1.0, "V/nm")
ion_field.to('atomic_unit_of_electric_field')

So ~0.002 times less than the proton-electron field - which makes sense. Protons and electrons are close together and if we had huge fields from ions, we'd rip electrons out of atoms all the time.

### Energy conversions:

1 Hartree is the "atomic unit" of energy

In [ ]:
energy = Q_(1.0, 'hartree')
energy.to('eV')

In [ ]:
energy.to('kcal') * 6.023e23/ureg.mol

### Spectroscopy conversions:


In [ ]:
q = 500 * ureg.nm
# q.to('Hz') # this will give a 'DimensionalityError' because nm is a length and Hz is a frequency
q.to('Hz', 'spectroscopy') # too many sig figs

In [ ]:
# you don't really need the spectroscopy part
wavelength = 550 * ureg.nm
frequency = (ureg.speed_of_light / wavelength).to('Hz')
print(frequency)
print(frequency.to_compact()) # the to_compact() method is useful since it adds appropriate prefixes like kilo-, nano-

In [ ]:
# we see a peak in our IR spectra at 1600 wavenumbers
ir = 1600 * ureg.cm_1
wavelength = ir.to('nm', 'spectroscopy')
format(wavelength, '.0f')

-------
This notebook is from Prof. Geoffrey Hutchison, University of Pittsburgh
https://github.com/ghutchis/chem1000

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.